In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

### Loading the Model and Testing it

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

In [6]:
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

for model in client.models.list():
    print(model.name)

models/embedding-gecko-001
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/ge

In [7]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [8]:
message = [SystemMessage(content="You are a helpful assistant."), HumanMessage(content="What is the capital of India?")]

In [9]:
message2 = [SystemMessage(content="You are a helpful assistant and always come up with funny answers."), HumanMessage(content="What is the capital of India?")]

In [10]:
model.invoke(message)

AIMessage(content='The capital of India is **New Delhi**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--73c6385f-d930-4cb9-beae-9f58a655a880-0', usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 317, 'input_token_details': {'cache_read': 0}})

In [11]:
model.invoke(message2)

AIMessage(content="Oh, that's a *capital* question!\n\nThe capital of India is **New Delhi**.\n\nIt's right next to Old Delhi, which I assume is where all the vintage, artisanal traffic jams are located.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--1ee17b2b-67d2-469e-870f-6bcd13891315-0', usage_metadata={'input_tokens': 21, 'output_tokens': 46, 'total_tokens': 1125, 'input_token_details': {'cache_read': 0}})

### Use Output Parser

In [12]:
from langchain_core.output_parsers import StrOutputParser

In [13]:
parser = StrOutputParser()

In [14]:
response = model.invoke(message)

In [15]:
response2 = model.invoke(message2)

In [16]:
parser.invoke(response)

'The capital of India is **New Delhi**.'

In [17]:
parser.invoke(response2)

'Oh, that\'s an easy one! The capital of India is **New Delhi**.\n\nIt\'s the city that got the "new" in its name because "Old Delhi" was too busy perfecting street food recipes to deal with all the governmental paperwork. A wise choice, if you ask me.'

### Chaining with LCEL

In [18]:
chain = model | parser

In [19]:
chain.invoke(message)

'The capital of India is **New Delhi**.'

In [20]:
chain.invoke(message2)

'Oh, that\'s an easy one! The capital of India is **New Delhi**.\n\nIt\'s not to be confused with an "Old Deli," which is where you\'d find a questionable pastrami sandwich that\'s been sitting out all day.\n\nNew Delhi is the political hub, the home of spicy debates, and unofficially, the world capital of creatively using your car horn.'

### Prompt Templating

In [21]:
from langchain_core.prompts import ChatPromptTemplate

In [22]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "translate the following text to {language}"), ("user", "{text}")]
)

In [23]:
prompt = prompt_template.invoke({"language": "French", "text": "I started learning a new topic today, I want to be a good person"})

In [24]:
parser.invoke(model.invoke(prompt))

'Of course! Here are a few ways to translate that, from the most direct to slightly more nuanced options.\n\n**Most direct translation:**\n\n> **J\'ai commencé à apprendre un nouveau sujet aujourd\'hui, je veux être une bonne personne.**\n\n---\n\n**Slightly more fluid alternatives:**\n\n> **Aujourd\'hui, j\'ai commencé à apprendre un nouveau sujet et je veux devenir une bonne personne.**\n(This version uses "aujourd\'hui" at the beginning, which is common in French, and adds "et" (and) to connect the two ideas. It also uses "devenir" (to become), which is a very natural way to express this.)\n\n> **J\'ai commencé à étudier un nouveau thème aujourd\'hui. Je veux être quelqu\'un de bien.**\n(This version uses "étudier" (to study) and "quelqu\'un de bien" (someone good), which is another common way to say "a good person.")'

In [25]:
prompt.to_messages()

[SystemMessage(content='translate the following text to French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I started learning a new topic today, I want to be a good person', additional_kwargs={}, response_metadata={})]

### Chaining the Prompt Template, Model and Parser

In [26]:
chain = prompt_template | model | parser

In [27]:
chain.invoke({"language": "marathi", "text": "I am learning a new language today, I want to be a good person"})

'Of course! Here is the translation in Marathi.\n\n**मी आज एक नवीन भाषा शिकत आहे, मला एक चांगला माणूस व्हायचं आहे.**\n\n---\n\n**Roman Transliteration (for pronunciation):**\n\n*Mī āj ek navīn bhāṣā shikat āhe, malā ek chānglā māṇūs vhāycha āhe.*\n\n---\n\n**Breakdown of the translation:**\n\n*   **मी (Mī)** = I\n*   **आज (āj)** = today\n*   **एक नवीन भाषा (ek navīn bhāṣā)** = a new language\n*   **शिकत आहे (shikat āhe)** = am learning\n*   **मला (malā)** = I (used here to express a want/desire)\n*   **एक चांगला माणूस (ek chānglā māṇūs)** = a good person\n*   **व्हायचं आहे (vhāycha āhe)** = want to be / become'

In [28]:
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING=os.getenv("LANGCHAIN_TRACING")

In [29]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING"] = LANGCHAIN_TRACING